In [1]:
import torch.distributions.constraints as constraints
from sklearn.model_selection import train_test_split
import torch
from sklearn.datasets import load_digits
import pyro.distributions as dist
import pyro.optim as optim
import pyro
from pyro.infer import SVI, Trace_ELBO
from pyro.infer import Predictive
import warnings
warnings.filterwarnings("ignore")

dataset = load_digits()
x, y = dataset.data, dataset.target
x=torch.tensor(x).float()
X=torch.tensor((x-x.min())/(x.max()-x.min()))
y=torch.tensor(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

pyro.clear_param_store()
def model(x, y):
    w = pyro.sample("w", dist.Normal(torch.zeros(len(y),x.shape[1]), torch.ones(len(y), x.shape[1])))
    b = pyro.sample("b", dist.Normal(torch.zeros(len(y)),torch.ones(len(y))))
    yhat = torch.sigmoid(torch.matmul(x,w.T) + b)
    with pyro.plate("data",  x.shape[0]):
        y = pyro.sample("y", dist.Categorical(yhat), obs=y)
        

def guide(x,y):
    wscale = pyro.param("wscale", torch.rand(len(y),  x.shape[1]), constraint=constraints.positive)
    wloc = pyro.param("wloc", torch.rand(len(y), x.shape[1]))
    w = pyro.sample("w", dist.Normal(wloc, wscale))
    bloc = pyro.param("bloc", torch.rand(len(y)))
    bscale = pyro.param("bscale", torch.rand(len(y)),constraint=constraints.positive)
    b = pyro.sample("b", dist.Normal(bloc, bscale))
    
svi = SVI(model=model, guide=guide, loss=Trace_ELBO(), optim=optim.Adam({'lr' : 0.05}))
for step in range(100):
    loss = svi.step(X_train, y_train)/len(X_train)
w = pyro.get_param_store()["wloc"]
b = pyro.get_param_store()["bloc"]
def predict(x):
    out = torch.sigmoid(torch.matmul(x,w.T) + b)
    return torch.argmax(out,dim=0)
prediction=predict(X_test)